In [24]:
import os

print("Current working dir: ", os.getcwd())

Current working dir:  /Users/nicky/Desktop/nba-stats-prediction/notebooks


In [25]:
import sys
import os
import pandas as pd

# So we can import from src/
sys.path.append(os.path.abspath("../src"))

from data_loader import fetch_multiple_seasons, save_to_csv

# Fetch data from 2017–2023
df = fetch_multiple_seasons(2017, 2023)

# Look at the first few rows
df.head()



Fetching season 2017-18...
Fetching season 2018-19...
Fetching season 2019-20...
Fetching season 2020-21...
Fetching season 2021-22...
Fetching season 2022-23...
Fetching season 2023-24...


,PLAYER_ID,PLAYER_NAME,NICKNAME,TEAM_ID,TEAM_ABBREVIATION,AGE,GP,W,L,W_PCT,...,BLKA_RANK,PF_RANK,PFD_RANK,PTS_RANK,PLUS_MINUS_RANK,NBA_FANTASY_PTS_RANK,DD2_RANK,TD3_RANK,WNBA_FANTASY_PTS_RANK,SEASON
0,201166,Aaron Brooks,Aaron,1610612750,MIN,33.0,32,19,13,0.594,...,427,447,472,459,422,484,228,32,479,2017-18
1,203932,Aaron Gordon,Aaron,1610612753,ORL,22.0,58,19,39,0.328,...,78,197,49,43,375,43,30,32,40,2017-18
2,1626151,Aaron Harrison,Aaron,1610612742,DAL,23.0,9,2,7,0.222,...,485,16,229,274,534,269,228,32,270,2017-18
3,1628935,Aaron Jackson,Aaron,1610612745,HOU,32.0,1,0,1,0.000,...,25,1,58,223,538,334,228,32,294,2017-18
4,1627846,Abdel Nader,Abdel,1610612738,BOS,24.0,48,33,15,0.688,...,348,439,420,432,420,447,228,32,439,2017-18


In [26]:
import os

output_path = os.path.abspath("../data/player_stats.csv")
print("Saving to:", output_path)

os.makedirs(os.path.dirname(output_path), exist_ok=True)

df.to_csv(output_path, index=False)
print("Saved")

Saving to: /Users/nicky/Desktop/nba-stats-prediction/data/player_stats.csv
Saved


In [27]:
import sys
import os
import pandas as pd
sys.path.append(os.path.abspath("../src"))

from feature_engineering import prepare_training_data

# Load the data
df = pd.read_csv("../data/player_stats.csv")

# 10 target stats
target_columns = ['PTS', 'AST', 'REB', 'ST'
'L', 'BLK', 'TOV', 'FG_PCT', 'FT_PCT', 'FG3_PCT', 'FG3M']

#prepare dataset
model_df = prepare_training_data(df, target_columns)

model_df.head()

,PLAYER_ID,PLAYER_NAME,NICKNAME,TEAM_ID,TEAM_ABBREVIATION,AGE,GP,W,L,W_PCT,...,NEXT_PTS,NEXT_AST,NEXT_REB,NEXT_STL,NEXT_BLK,NEXT_TOV,NEXT_FG_PCT,NEXT_FT_PCT,NEXT_FG3_PCT,NEXT_FG3M
0,1713,Vince Carter,Vince,1610612758,SAC,41.0,58,20,38,0.345,...,5.4,1.2,2.6,0.7,0.4,0.6,0.403,0.757,0.345,1.0
1,1713,Vince Carter,Vince,1610612737,ATL,42.0,76,25,51,0.329,...,7.4,1.1,2.6,0.6,0.4,0.6,0.419,0.712,0.389,1.6
2,1713,Vince Carter,Vince,1610612737,ATL,43.0,60,19,41,0.317,...,5.0,0.8,2.1,0.4,0.4,0.6,0.352,0.793,0.302,1.0
3,1717,Dirk Nowitzki,Dirk,1610612742,DAL,40.0,77,24,53,0.312,...,12.0,1.6,5.7,0.6,0.6,0.7,0.456,0.898,0.409,1.8
4,1717,Dirk Nowitzki,Dirk,1610612742,DAL,41.0,51,17,34,0.333,...,7.3,0.7,3.1,0.2,0.4,0.4,0.359,0.780,0.312,1.3


In [28]:
# drop any rows where NEXT stat columns are missing
model_df_clean = model_df.dropna(subset=[f'NEXT_{col}' for col in target_columns])

# check how many rows are left
print("Rows before cleaning:", len(model_df))
print("Rows after cleaning:", len(model_df_clean))

model_df_clean = model_df_clean.drop_duplicates(subset=['PLAYER_ID', 'SEASON'])

#define X features and y targets

leakage_cols = ['PTS', 'PTS_RANK', 'NBA_FANTASY_PTS', 'NBA_FANTASY_PTS_RANK']

X = model_df_clean.drop(columns=[
    'PLAYER_ID', 'PLAYER_NAME', 'NICKNAME', 'TEAM_ID', 'TEAM_ABBREVIATION', 'SEASON'
] + leakage_cols + list(f'NEXT_{col}' for col in target_columns))

y = model_df_clean[[f'NEXT_{col}' for col in target_columns]]

X.head(), y.head()

Rows before cleaning: 3855
Rows after cleaning: 3855


(    AGE  GP   W   L  W_PCT   MIN  FGM  FGA  FG_PCT  FG3M  ...  TOV_RANK  \
 0  41.0  58  20  38  0.345  17.7  2.0  4.9   0.403   1.0  ...       363   
 1  42.0  76  25  51  0.329  17.5  2.6  6.2   0.419   1.6  ...       355   
 2  43.0  60  19  41  0.317  14.6  1.8  5.1   0.352   1.0  ...       399   
 3  40.0  77  24  53  0.312  24.7  4.5  9.8   0.456   1.8  ...       357   
 4  41.0  51  17  34  0.333  15.6  2.6  7.4   0.359   1.3  ...       449   
 
    STL_RANK  BLK_RANK  BLKA_RANK  PF_RANK  PFD_RANK  PLUS_MINUS_RANK  \
 0       189       153        466      275       303              335   
 1       242       221        446      239       295              418   
 2       361       184        447      328       438              396   
 3       265       111        468      205       210              296   
 4       472       222        491      335       374              331   
 
    DD2_RANK  TD3_RANK  WNBA_FANTASY_PTS_RANK  
 0       228        32                    310  
 1    

In [29]:
import numpy as np

# Make a copy to avoid modifying the original directly
fe_df = model_df_clean.copy()

# Add age squared to model peak age behavior
fe_df["AGE_SQUARED"] = fe_df["AGE"] ** 2

# Add per-minute stats for major box score categories
for stat in ['PTS', 'AST', 'REB', 'STL', 'BLK', 'TOV']:
    fe_df[f"{stat}_PER_MIN"] = fe_df[stat] / fe_df['GP'].replace(0, np.nan)

# Turn SEASON like "2018-19" → numeric value (e.g. 2018)
fe_df["SEASON_NUM"] = fe_df["SEASON"].apply(lambda x: int(x.split("-")[0]))

# Check a sample
fe_df[["AGE", "AGE_SQUARED", "PTS_PER_MIN", "SEASON", "SEASON_NUM"]].head()


,AGE,AGE_SQUARED,PTS_PER_MIN,SEASON,SEASON_NUM
0,41.0,1681.0,0.093103,2017-18,2017
1,42.0,1764.0,0.097368,2018-19,2018
2,43.0,1849.0,0.083333,2019-20,2019
3,40.0,1600.0,0.155844,2017-18,2017
4,41.0,1681.0,0.143137,2018-19,2018


In [30]:
# Columns to exclude (identifiers, targets, ranks, fantasy)
excluded_cols = [
    'PLAYER_ID', 'PLAYER_NAME', 'NICKNAME', 'TEAM_ID',
    'TEAM_ABBREVIATION', 'SEASON'
] + list(f'NEXT_{col}' for col in target_columns) + [
    col for col in fe_df.columns if 'RANK' in col or 'FANTASY' in col
]

# Final X and y
X = fe_df.drop(columns=excluded_cols)
y = fe_df[[f'NEXT_{col}' for col in target_columns]]

X.head()


,AGE,GP,W,L,W_PCT,MIN,FGM,FGA,FG_PCT,FG3M,...,DD2,TD3,AGE_SQUARED,PTS_PER_MIN,AST_PER_MIN,REB_PER_MIN,STL_PER_MIN,BLK_PER_MIN,TOV_PER_MIN,SEASON_NUM
0,41.0,58,20,38,0.345,17.7,2.0,4.9,0.403,1.0,...,0,0,1681.0,0.093103,0.020690,0.044828,0.012069,0.006897,0.010345,2017
1,42.0,76,25,51,0.329,17.5,2.6,6.2,0.419,1.6,...,0,0,1764.0,0.097368,0.014474,0.034211,0.007895,0.005263,0.007895,2018
2,43.0,60,19,41,0.317,14.6,1.8,5.1,0.352,1.0,...,0,0,1849.0,0.083333,0.013333,0.035000,0.006667,0.006667,0.010000,2019
3,40.0,77,24,53,0.312,24.7,4.5,9.8,0.456,1.8,...,5,0,1600.0,0.155844,0.020779,0.074026,0.007792,0.007792,0.009091,2017
4,41.0,51,17,34,0.333,15.6,2.6,7.4,0.359,1.3,...,1,0,1681.0,0.143137,0.013725,0.060784,0.003922,0.007843,0.007843,2018


In [31]:
selected_features = [
    'AGE', 'AGE_SQUARED', 'MIN', 'GP', 'W_PCT', 'PLUS_MINUS',
    'FG_PCT', 'FT_PCT', 'FG3_PCT', 'SEASON_NUM'
]

X = fe_df[selected_features]



In [32]:
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, r2_score

# Predict NEXT_PTS 
target = 'NEXT_PTS'

# train/test split
X_train, X_test, y_train, y_test = train_test_split(X, y[target], test_size=0.2, random_state=42)

# train linear regression model
model = LinearRegression()
model.fit(X_train, y_train)

#predict
y_pred = model.predict(X_test)

#Evaluate
print("MSE:", mean_squared_error(y_test, y_pred))
print("R2:", r2_score(y_test, y_pred))

MSE: 8.733507310916725
R2: 0.7976888191597531


In [33]:
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, r2_score

results = {}

for target in y.columns:
    X_train, X_test, y_train, y_test = train_test_split(X, y[target], test_size=0.2, random_state=42)
    model = LinearRegression()
    model.fit(X_train, y_train)
    y_pred = model.predict(X_test)

    mse = mean_squared_error(y_test, y_pred)
    r2 = r2_score(y_test, y_pred)

    results[target] = {
        "mse": mse,
        "r2": r2
    }

# Show results
for stat, metrics in results.items():
    print(f"{stat}: MSE={metrics['mse']:.4f}, R2={metrics['r2']:.4f}")


NEXT_PTS: MSE=8.7335, R2=0.7977
NEXT_AST: MSE=1.5797, R2=0.5476
NEXT_REB: MSE=2.4318, R2=0.5900
NEXT_STL: MSE=0.0691, R2=0.5822
NEXT_BLK: MSE=0.1025, R2=0.3112
NEXT_TOV: MSE=0.2673, R2=0.6351
NEXT_FG_PCT: MSE=0.0000, R2=1.0000
NEXT_FT_PCT: MSE=0.0000, R2=1.0000
NEXT_FG3_PCT: MSE=0.0000, R2=1.0000
NEXT_FG3M: MSE=0.2576, R2=0.6598
